In [2]:
import numpy as np
import pandas as pd

In [125]:
class MLP(object):
    model = None

    @staticmethod
    def f(net):
        return ( 1/ (1+ np.exp(-net)) )

    @staticmethod
    def df_dnet(f_net):
        return ( f_net * (1 - f_net) )

    def __init__(self, input_length=2, hidden_length=[3, 3], output_length=1, activation_function=f , d_activation_function=df_dnet):
        hidden_layer = list()
        previous_length = input_length
        for layer_length in hidden_length:
            hidden_layer.append(np.random.rand(layer_length, previous_length+1) - 0.5)
            previous_length = layer_length

        self.model = {
            'input_length': input_length, 
            'hidden_length': hidden_length, 
            'output_length': output_length, 
            'activation_function': activation_function.__func__, 
            'd_activation_function': d_activation_function.__func__,
            'hidden': hidden_layer,
            'output': (np.random.rand(output_length, previous_length+1) - 0.5),
        }

    def forward(self, x):
        # Recuperando valores do modelo
        hidden = self.model['hidden']
        hidden_length = len(self.model['hidden_length'])
        output = self.model['output']
        f = self.model['activation_function']
        df = self.model['d_activation_function']

        # Camadas Escondidas
        net_h = [None] * hidden_length
        f_net_h = [None] * hidden_length
        df_net_h = [None] * hidden_length
        
        
        # Adicionando 1 para multiplicar o Theta.
        previous_layer = np.pad(x, (0, 1), 'constant', constant_values=(1))
        for i in range(hidden_length):
            net_h[i] = np.sum(np.multiply(hidden[i], previous_layer), axis=1)
            f_net_h[i] = f(net_h[i])
            df_net_h[i] = df(f_net_h[i])
            previous_layer = np.pad(f_net_h[i], (0, 1), 'constant', constant_values=(1))

            
        # Camada de Saída
        net_o = np.sum(np.multiply(output, previous_layer), axis=1)
        f_net_o = f(net_o)
        df_net_o = df(f_net_o)

        # Retornando valores do forward.
        return {
            'net_h': net_h,
            'f_net_h': f_net_h,
            'df_net_h': df_net_h,
            'net_o': net_o,
            'f_net_o': f_net_o,
            'df_net_o': df_net_o,
        }

    def backpropagation(self, X, Y, eta=0.1, threshold=1e-3, alpha=0,max_inter = 200000):
        squaredError = 2*threshold
        hidden_length = len(self.model['hidden_length'])
        counter = 0
        
        dE2_dw_o_t = 0
        dE2_dw_h_t = [0] * hidden_length
        while(squaredError > threshold and counter < max_inter):
            squaredError = 0
            # Pra cada valor do conjunto de dados
            for x, y in zip(X, Y):
                # Calculando saída
                results = self.forward(x)
                #  Calculando o erro
                error = (y - results['f_net_o'])                
                squaredError += np.sum(np.power(error, 2))
                

                # Backwards camada de saída
                h = hidden_length - 1
                delta_o = error * results['df_net_o']
                f_net_h = np.pad(results['f_net_h'][h], (0, 1), 'constant', constant_values=(1))
                dE2_dw_o = np.multiply(np.array([-2*delta_o]).T, np.array([f_net_h]))
                
                # Backwards camada escondida
                delta_h = [None] * hidden_length
                dE2_dw_h = [None] * hidden_length
                delta = delta_o
                w_o_kj = self.model['output'][:,0:self.model['hidden_length'][h]] 

                # For reverso
                for i in reversed(range(1, hidden_length)):
                    # Cálculo das adaptações
                    delta_h[i] = np.array([results['df_net_h'][i]]) * np.dot(delta, w_o_kj)         
                    dE2_dw_h[i] = np.multiply(-2*delta_h[i].T,
                        np.pad(results['f_net_h'][i-1], (0, 1), 'constant', constant_values=(1)))
                    # Controlando as iterações
                    delta = delta_h[i]
                    w_o_kj = self.model['hidden'][i][:, 0:self.model['hidden_length'][i-1]]
                    
                delta_h[0] = np.array([results['df_net_h'][0]]) * np.dot(delta, w_o_kj)               
                dE2_dw_h[0] =  np.multiply(-2*delta_h[0].T,
                    np.pad(x, (0, 1), 'constant', constant_values=(1)))
                               
                
                # Aplicar adaptação na saída
                self.model['output'] = self.model['output'] - eta * dE2_dw_o - alpha*dE2_dw_o_t

                # Aplicar adaptação na escondida
                for i in range(hidden_length):
                    self.model['hidden'][i] = self.model['hidden'][i] - eta * dE2_dw_h[i] - alpha*dE2_dw_h_t[i]

                dE2_dw_o_t = dE2_dw_o
                dE2_dw_h_t = dE2_dw_h
                
            squaredError = squaredError / len(X) 
            counter += 1
            if(counter % 1000 == 0):
                print('error %.6lf - iter. %d' % (squaredError, counter))



In [167]:
class RBFN(object):
    def __init__(self, input_shape, hidden_shape, output_shape):
        self.input_shape = input_shape
        self.hidden_shape = hidden_shape
        self.sigmas = None
        self.centers = None
        self.mlp = MLP(input_length=self.hidden_shape,hidden_length=[1], output_length=output_shape)

    def _kernel_function(self, center, x, sigma):
        return np.exp(-(np.linalg.norm(center-x)**2)/(2*sigma**2) )
    
    
    def _kernel_layer_fit(self, X, centers, threshold=1e-5, sig_err=1e-5):
        ids = np.random.choice(X.shape[0], centers, replace=False)
        centroids = X[ids,]
        error = 2*threshold

        while (error > threshold):
            distances = []
            for c in centroids:
                distances.append(
                    np.apply_along_axis(lambda x, u: np.linalg.norm(x-u), 1, X, c)
                    )

            ids = np.argsort(np.array(distances).T)[:,0]
            error = 0
            for i in range(centers):
                old = centroids[i, ]
                centroids[i,] = np.mean(X[ids == i], 1)
                error = error + np.sqrt(np.sum(np.power(centroids[i,] - old, 2)))

        self.centers = centroids
        self.sigmas = [np.std(X[ids == i])+sig_err for i in range(centers)]
        self.centroid_to_x = ids


    def fit(self, X, Y, eta=0.1, alpha=0):
        self._kernel_layer_fit(X, self.hidden_shape)
        transformed_X = []
        for i in range(self.hidden_shape):
            transformed_X.append(np.apply_along_axis(self._kernel_function, 1, X, self.centers[i], self.sigmas[i]))

        transformed_X = np.array(transformed_X).T
        self.mlp.backpropagation(transformed_X, Y, eta=eta, alpha=alpha)

        
    def predict(self, x):
        transformed_x = []
        for i in range(self.hidden_shape):
            transformed_x.append(self._kernel_function(x, self.centers[i], self.sigmas[i]))
        return self.mlp.forward(transformed_x)['f_net_o'][0]


xor_x = np.array([0, 0, 0, 1, 1, 0, 1, 1]).reshape((4,2))
xor_y = np.array([x[0]^x[1] for x in xor_x])

rbfn = RBFN(2, 3, 1)
rbfn.fit(xor_x, xor_y, eta=0.8)

    
for x, y in zip(xor_x, xor_y):
    print(rbfn.predict(x))

0.03294511906980418
0.9713399885299672
0.9713399885299672
0.03543067688421307


In [49]:
A = 
x = np.arange(100)
A(x, 4)

array([ 4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,
        9., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21.,
       22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34.,
       35., 36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47.,
       48., 49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60.,
       61., 62., 63., 64., 65., 66., 67., 68., 69., 70., 71., 72., 73.,
       74., 75., 76., 77., 78., 79., 80., 81., 82., 83., 84., 85., 86.,
       87., 88., 89., 90., 91., 92., 93., 94., 95.])